<a href="https://colab.research.google.com/github/R267/Pryklad_analyz_exam/blob/main/task7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
razamh_pubg_dataset_path = kagglehub.dataset_download('razamh/pubg-dataset')

print('Data source import complete.')


100%|██████████| 64.5M/64.5M [00:01<00:00, 61.5MB/s]

Extracting files...


Data source import complete.


# PUBG Дослідницький аналіз даних (EDA)










Завдання

У грі PUBG у кожному матчі (matchId) може брати участь до 100 гравців. Гравці можуть бути об’єднані в команди (groupId), які на кінці гри отримують ранжування (winPlacePerc) залежно від того, скільки інших команд залишаються живими на момент їх вибування. Під час гри гравці можуть збирати різні боєприпаси, відроджувати поранених, але не вибитих товаришів (knocked), керувати транспортом, плавати, бігати, стріляти і стикатися з усіма наслідками — наприклад, падінням з великої висоти або випадковим наїздом на самого себе, що може призвести до вибування.

Вам надаються анонімізовані статистичні дані по багатьом іграм PUBG, у яких кожен рядок містить статистику одного гравця після матчу. Дані охоплюють різні типи матчів: соло, дуети, сквади та кастомні. Немає гарантії, що в кожному матчі було рівно 100 гравців або максимум 4 гравці в команді.











Розуміння набору даних
Інформація про набір даних

Ознаки

Ознака	Опис
DBNOs	Кількість ворогів, які були поранені (knocked).
assists	Кількість ворогів, пошкоджених цим гравцем, які були вбиті товаришами по команді.
boosts	Кількість використаних підсилюючих предметів.
damageDealt	Загальна завдана шкода. Примітка: Шкода, завдана собі, віднімається.
headshotKills	Кількість вбитих ворогів із пострілом у голову.
heals	Кількість використаних лікувальних предметів.
Id	Ідентифікатор гравця.
killPlace	Рейтинг за кількістю вбитих ворогів у матчі.
killPoints	Зовнішній рейтинг гравця, заснований на вбивствах (аналог рейтингу Ело, де враховуються лише вбивства). Якщо у rankPoints є значення, відмінне від -1, то будь-яке значення 0 у killPoints слід вважати «відсутнім».
killStreaks	Максимальна кількість ворогів, вбитих за короткий проміжок часу.
kills	Кількість вбитих ворогів.
longestKill	Найдовша відстань між гравцем і вбитим ворогом у момент смерті. Може бути оманливою, адже поранення гравця і від’їзд можуть збільшити цей показник.
matchDuration	Тривалість матчу в секундах.
matchId	Ідентифікатор матчу. У тренувальному та тестовому наборі немає спільних матчів.
matchType	Режим гри (рядок). Стандартні режими: “solo”, “duo”, “squad”, “solo-fpp”, “duo-fpp”, “squad-fpp”; інші — події або кастомні матчі.
rankPoints	Рейтинг гравця на кшталт Ело. Рейтинг є нестабільним і буде застарілим у наступній версії API, тому використовуйте з обережністю. Значення -1 означає «відсутній».
revives	Кількість разів, коли гравець відроджував товаришів по команді.
rideDistance	Загальна відстань, пройдена на транспорті (метри).
roadKills	Кількість вбивств під час керування транспортом.
swimDistance	Загальна відстань, пропливена плавцем (метри).
teamKills	Кількість разів, коли гравець вбив товариша по команді.
vehicleDestroys	Кількість знищених транспортних засобів.
walkDistance	Загальна відстань, пройдена пішки (метри).
weaponsAcquired	Кількість піднятої зброї.
winPoints	Зовнішній рейтинг, заснований на перемогах (аналог Ело, де враховується лише перемога). Якщо у rankPoints є значення, відмінне від -1, то будь-яке значення 0 у winPoints слід вважати «відсутнім».
groupId	Ідентифікатор групи в межах матчу. Якщо одна й та сама група гравців грає в різних матчах, у кожному з них вона матиме різний groupId.
numGroups	Кількість груп, для яких є дані в матчі.
maxPlace	Найгірше місце, зафіксоване в матчі. Може не збігатися з numGroups, адже іноді деякі позиції пропускаються.
winPlacePerc	Цільова змінна для прогнозування. Відсоткове ранжування за зайнятим місцем: 1 — перше місце, 0 — останнє. Обчислюється на основі maxPlace, а не numGroups, тому можливі пропуски в даних за місцями.

###  Імпорт необхідних бібліотек

Наступний код написаний на Python 3.x. Бібліотеки надають готовий функціонал для виконання необхідних завдань.










In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [3]:
# loading the dataset in the variable called 'df' and setting the index column as 'Id'

df= pd.read_csv('/content/data.csv', index_col='Id')

In [4]:
df.head()  # To view top 5 entries in the dataset.

# in order to view bottom 5 entries, we can do
#df.tail()

#in order to view more than 5 entries, we can enter any integer value into '()'.
#Ex: df.head(10) or df.tail(15), etc

,Unnamed: 0,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
Id,,,,,,,,,,,,,,,,,,,,,
a3e3cea9f2e314,2093518,c9f6eaa81245b5,a3d8693390584c,0,0,25.93,0,0,0,84,...,0,0.0,0,0.0,0,0,151.30,1,1525,0.0667
43e4be7ad72cc7,3701672,521836de101ee8,b65de9055327e0,0,0,36.71,0,0,0,57,...,0,0.0,0,0.0,0,0,1918.00,7,1516,0.5862
9a68690e31fdff,2059784,4a72ffa2cebd90,d6aad3f9830e60,0,1,47.73,0,0,0,50,...,0,4088.0,0,0.0,0,0,820.00,7,0,0.8105
b147e1bd448fc4,1302036,9a8991656b3fea,d931c0932d8aca,0,0,0.00,0,0,0,56,...,1,0.0,0,0.0,0,0,38.81,0,1515,0.5556
d818b4edd59612,297180,eece87c8b846b3,ec2b5ed94baae3,0,2,100.00,0,0,3,31,...,0,918.4,0,0.0,0,0,1721.00,5,0,0.6429


In [5]:
# Now, we let us see all the column names.
df.columns

Index(['Unnamed: 0', 'groupId', 'matchId', 'assists', 'boosts', 'damageDealt',
       'DBNOs', 'headshotKills', 'heals', 'killPlace', 'killPoints', 'kills',
       'killStreaks', 'longestKill', 'matchDuration', 'matchType', 'maxPlace',
       'numGroups', 'rankPoints', 'revives', 'rideDistance', 'roadKills',
       'swimDistance', 'teamKills', 'vehicleDestroys', 'walkDistance',
       'weaponsAcquired', 'winPoints', 'winPlacePerc'],
      dtype='object')

In [6]:
# We can see that the column named 'Unnamed: 0' can be removed from the dataset as it is not clear, what it represents.

df= df.drop(['Unnamed: 0'], axis=1)  # If axis=0, it consitutes row operation. Since we have to remove the column, we do axis=1.

In [7]:
df.head(2)

,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
Id,,,,,,,,,,,,,,,,,,,,,
a3e3cea9f2e314,c9f6eaa81245b5,a3d8693390584c,0,0,25.93,0,0,0,84,1437,...,0,0.0,0,0.0,0,0,151.3,1,1525,0.0667
43e4be7ad72cc7,521836de101ee8,b65de9055327e0,0,0,36.71,0,0,0,57,1225,...,0,0.0,0,0.0,0,0,1918.0,7,1516,0.5862


In [8]:
# Now let us check for the shape of the dataset and also that are any null values present in our dataset.
# For that,

print('shape of the dataset=', df.shape)

print(' \nThe null count of each column of the dataset are as follows:')
df.isnull().sum()

shape of the dataset= (1111742, 28)
 
The null count of each column of the dataset are as follows:


,0
groupId,0
matchId,0
assists,0
boosts,0
damageDealt,0
DBNOs,0
headshotKills,0
heals,0
killPlace,0
killPoints,0


З наведених вище даних видно, що розмірність набору даних становить (1111742, 28). Це означає, що набір містить 1 111 742 рядки та 28 стовпців.

Пізніше ми помічаємо, що відсутніх значень немає, окрім одного — у стовпці winPlacePerc є 1 пропущене значення.

In [9]:
# To view the null row from the dataset:

df[df['winPlacePerc'].isnull() == True]

,groupId,matchId,assists,boosts,damageDealt,DBNOs,headshotKills,heals,killPlace,killPoints,...,revives,rideDistance,roadKills,swimDistance,teamKills,vehicleDestroys,walkDistance,weaponsAcquired,winPoints,winPlacePerc
Id,,,,,,,,,,,,,,,,,,,,,
f70c74418bb064,12dfbede33f92b,224a123c53e008,0,0,0.0,0,0,0,1,0,...,0,0.0,0,0.0,0,0,0.0,0,0,NaN


In [10]:
# Function to identify numeric features:

def numeric_features(dataset):
    numeric_col = dataset.select_dtypes(include=np.number).columns.tolist()
    return dataset[numeric_col].head()

numeric_columns = numeric_features(df)
print("Numerical Features:")
print(numeric_columns)

print("===="*20)




# Function to identify categorical features:

def categorical_features(dataset):
    categorical_col = dataset.select_dtypes(exclude=np.number).columns.tolist()
    return dataset[categorical_col].head()

categorical_columns = categorical_features(df)
print("Categorical Features:")
print(categorical_columns)

print("===="*20)



# Function to check the datatypes of all the columns:

def check_datatypes(dataset):
    return dataset.dtypes

print("Datatypes of all the columns:")
check_datatypes(df)

Numerical Features:
                assists  boosts  damageDealt  DBNOs  headshotKills  heals  \
Id                                                                          
a3e3cea9f2e314        0       0        25.93      0              0      0   
43e4be7ad72cc7        0       0        36.71      0              0      0   
9a68690e31fdff        0       1        47.73      0              0      0   
b147e1bd448fc4        0       0         0.00      0              0      0   
d818b4edd59612        0       2       100.00      0              0      3   

                killPlace  killPoints  kills  killStreaks  ...  revives  \
Id                                                         ...            
a3e3cea9f2e314         84        1437      0            0  ...        0   
43e4be7ad72cc7         57        1225      0            0  ...        0   
9a68690e31fdff         50           0      0            0  ...        0   
b147e1bd448fc4         56        1286      0            0  ...   

,0
groupId,object
matchId,object
assists,int64
boosts,int64
damageDealt,float64
DBNOs,int64
headshotKills,int64
heals,int64
killPlace,int64
killPoints,int64


### Виявлення викидів у числових стовпцях

Викиди — це спостереження, які значно віддалені від більшості інших даних у наборі, і їх можна математично визначати різними способами.

Один із методів визначення викидів: викидами вважаються точки даних, що виходять за межі (третього квартиля + 1.5×IQR) зверху та (першого квартиля - 1.5×IQR) знизу.

Наведена нижче функція приймає DataFrame і виводить кількість викидів для кожної числової ознаки на основі вказаного правила IQR.

In [12]:
def detect_outliers(df):
    cols = list(df)
    outlier_list = []  # список для накопичення результатів

    for column in cols:
        if column in df.select_dtypes(include=np.number).columns:
            q1 = df[column].quantile(0.25)
            q3 = df[column].quantile(0.75)
            iqr = q3 - q1
            fence_low = q1 - (1.5 * iqr)
            fence_high = q3 + (1.5 * iqr)
            num_outliers = df.loc[(df[column] < fence_low) | (df[column] > fence_high)].shape[0]
            outlier_list.append({'Feature': column, 'Number of Outliers': num_outliers})

    outliers = pd.DataFrame(outlier_list)
    return outliers

detect_outliers(df)


,Feature,Number of Outliers
0,assists,194611
1,boosts,35533
2,damageDealt,54026
3,DBNOs,73607
4,headshotKills,187930
5,heals,81625
6,killPlace,0
7,killPoints,0
8,kills,129163
9,killStreaks,13391


### Спостереження:
Відповідно до методології IQR, більшість колонок містять викиди (outliers).

## EDA та Візуалізація Даних
Exploratory Data Analysis (EDA) — це підхід до аналізу наборів даних шляхом узагальнення їх основних характеристик за допомогою візуалізацій. Процес EDA є важливим кроком перед побудовою моделі, адже допомагає розкрити різні інсайти, які згодом відіграють ключову роль у створенні надійної алгоритмічної моделі.

## Уніваріантний аналіз
Уніваріантний аналіз означає аналіз однієї змінної. Він переважно описує характеристики цієї змінної.


#### Кількість ворогів, яких вбив гравець:
Це кількість ворогів-гравців, яких вбив кожен гравець.



In [13]:
# Summary statistics for the number of kills
print('The average person kills {:.4f} players'.format(df['kills'].mean()))
print('50% of people have ',df['kills'].quantile(0.50),' kills or less')
print('75% of people have ',df['kills'].quantile(0.75),' kills or less')
print('99% of people have ',df['kills'].quantile(0.99),' kills or less')
print('while the most kills recorded in the data is', df['kills'].max())

The average person kills 0.9255 players
50% of people have  0.0  kills or less
75% of people have  1.0  kills or less
99% of people have  7.0  kills or less
while the most kills recorded in the data is 72


Інший спосіб подання цієї статистичної інформації — у вигляді графіка, як показано нижче. Тут зображено кількість гравців, які зробили 1, 2, 3, ... 8 і більше вбивств у грі!



In [ ]:
data = df.copy()
data.loc[data['kills'] > data['kills'].quantile(0.99)] = '8+'
plt.figure(figsize=(20,15))
sns.countplot(data['kills'].astype('str').sort_values())
plt.title('Kill Count',fontsize=15)
plt.xlabel('Kills', fontsize=15)
plt.ylabel('Count',fontsize=13)
plt.show()

#### Максимальна кількість ворогів, вбитих за короткий час
Це кількість ворогів-гравців, вбитих кожним гравцем за короткий проміжок часу.

In [4]:
# Summary statistics for the number of kills
print('The average person kills {:.4f} players in a short time'.format(df['killStreaks'].mean()))
print('50% of people have ',df['killStreaks'].quantile(0.50),' kills or less in a short time')
print('75% of people have ',df['killStreaks'].quantile(0.75),' kills or less in a short time')
print('99% of people have ',df['killStreaks'].quantile(0.99),' kills or less in a short time')
print('While the most kills in a row recorded in the data is', df['killStreaks'].max())

NameError: name 'df' is not defined

Інший спосіб подання цієї статистичної інформації — у вигляді графіка, як показано нижче. Тут наведено графік кількості гравців, які зробили 1, 2, 3, 4 і більше послідовних вбивств (killStreaks) у грі!



In [3]:
data = df.copy()
data.loc[data['killStreaks'] > data['killStreaks'].quantile(0.99)] = '4+'
plt.figure(figsize=(20,15))
sns.countplot(data['killStreaks'].astype('str').sort_values())
plt.title('Kill Count',fontsize=15)
plt.xlabel('killStreaks', fontsize=15)
plt.ylabel('Count',fontsize=13)
plt.show()

NameError: name 'df' is not defined

#### Робота зі стовпцем 'matchType'


In [17]:
# To check how many unique values are present in this categorical column:

df['matchType'].value_counts()

,count
matchType,
squad-fpp,438781
duo-fpp,249927
squad,156573
solo-fpp,133854
duo,78039
solo,45679
normal-squad-fpp,4286
crashfpp,1581
normal-duo-fpp,1400


In [18]:
# To plot the above insights in form of 'countplot'

plt.figure(figsize=(20,15))
sns.countplot(df['matchType'], )
plt.title('Match Type',fontsize=15)
plt.xlabel('Match Type', fontsize=15)
plt.ylabel('Count',fontsize=13)
plt.show()

KeyboardInterrupt: 

Error in callback <function _draw_all_if_interactive at 0x798f7cc86e80> (for post_execute):


KeyboardInterrupt: 

Error in callback <function flush_figures at 0x798f7cc84ae0> (for post_execute):


KeyboardInterrupt: 

##### Спостереження:
З наведеного графіка очевидно, що найпопулярнішим типом матчів є squad-fpp.

Найменш популярним типом матчів є normal-duo.

#### Шкода ворогам
Ми бачили, що більшість гравців не можуть вбити нікого, тож можливо вони завдають хоча б якусь шкоду ворогам.

In [ ]:
data = df.copy()

# Keep only those players that didn't kill anyone
data = data[data['kills']==0]
plt.figure(figsize=(15,10))
plt.title('Damage Dealt by 0 killers',fontsize=15)
sns.distplot(data['damageDealt'])
plt.xlabel('Damage Dealt', fontsize=15)
plt.ylabel('Density',fontsize=13)
plt.show()

##### Спостереження:
Тут ми бачимо розподіл кількості шкоди, яку завдають гравці, які нікого не вбивають. Більшість гравців завдають небагато шкоди, ймовірно, це новачки, які вивчають керування та знайомляться з грою, у той час як їх постійно переможуть більш досвідчені гравці.



#### Давайте подивимось на тривалість матчів для всіх переможців.



In [ ]:
# Keep only the players that won the match
data = df[df['winPlacePerc'] == 1]

plt.figure(figsize=(15,10))
plt.title('Match duration for winners',fontsize=15)
sns.distplot(data['matchDuration'])
plt.xlabel('Match Duration', fontsize=15)
plt.ylabel('Density',fontsize=13)
plt.show()

##### Спостереження:
Схоже, що тривалість матчу не впливає на показник winPlacePerc. Очевидно, що виграти гру можна навіть за трохи більше ніж 2 хвилини, але найчастіше перемога припадає на приблизно 1400 або 1850 секунд.

Тривалість матчу навряд чи буде корисною ознакою для передбачення winPlacePerc.

### Двовимірний аналіз
Двовимірний аналіз полягає у вивченні взаємозв’язку між двома змінними одночасно.

Давайте розглянемо ці дані і перевіримо, чи існує кореляція з нашою цільовою змінною winPlacePerc.

Кореляція між двома випадковими змінними описує статистичний зв’язок, що фактично означає, наскільки близькі ці дві змінні до лінійної залежності. Кореляція може набувати значень від -1 до 1:

Кореляція 1 означає, що змінні ідеально корелюють.

Кореляція 0 означає відсутність кореляції між змінними.

Кореляція -1 означає, що змінні ідеально негативно корелюють.

In [ ]:
plt.figure(figsize=(15,10))
sns.jointplot(x='winPlacePerc', y='killStreaks', data=df, color='b')
# plt.title('Win place vs Kill Streaks')
plt.xlabel('Win Place Prec', fontsize=15)
plt.ylabel('Kill streaks',fontsize=13)
plt.show()

In [ ]:
sns.jointplot(x='winPlacePerc', y='damageDealt', data=df, color='b')

Тут спостерігається помірна кореляція між завданою шкодою ворогам і показником winPlacePerc.



#### Кількість разів, коли гравець вбив товариша по команді
Це кількість випадків, коли член команди вбив іншого гравця зі своєї ж команди.

In [ ]:
# Summary statistics for the number of kills
print('The average person kills {:.4f} players on their own team'.format(df['teamKills'].mean()))
print('50% of people have killed ',df['teamKills'].quantile(0.50),' team players')
print('75% of people have killed ',df['teamKills'].quantile(0.75),' team players')
print('99% of people have killed ',df['teamKills'].quantile(0.99),' team players')
print('while the most kills recorded in the data is', df['teamKills'].max())

In [ ]:
sns.jointplot(x='winPlacePerc', y='teamKills', data=df, ratio=3, color='r')

#### Загальна пройдена дистанція
Це не є наявною ознакою в даних, але ми можемо поєднати різні ознаки, що відповідають за відстань, щоб сформувати показник загальної пройденої дистанції та перевірити, чи має він прогностичну силу для нашої цільової змінної.

In [ ]:
# Create a new feature for total distance travelled
data = df[['winPlacePerc']].copy()
data['totalDistance'] = df['walkDistance'] + df['rideDistance'] + df['swimDistance']

# Summary statistics for the total distance travelled
print('The average person travelled {:.2f} m'.format(data['totalDistance'].mean()))
print('25% of people have travelled {:.2f} m or less'.format(data['totalDistance'].quantile(0.25)))
print('50% of people have travelled {:.2f} m or less'.format(data['totalDistance'].quantile(0.50)))
print('75% of people have travelled {:.2f} m or less'.format(data['totalDistance'].quantile(0.75)))
print('99% of people have travelled {:.2f} m or less'.format(data['totalDistance'].quantile(0.99)))
print('The longest distance travelled in the data is {:.2f} m'.format(data['totalDistance'].max()))

In [ ]:
sns.jointplot(x='winPlacePerc', y='totalDistance', data=data, ratio=3, color='r')


##### Спостереження:
Існує досить сильна кореляція між загальною пройденою дистанцією та перемогою, хоча більшість цієї кореляції може пояснюватися сильною кореляцією між ходьбою (walking distance) та показником перемоги (winPlacePerc).

Цікаво, що, здається, гравець, який пройшов найдовшу відстань — понад 41 км за одну гру, — не виграв матч.

#### Вплив лікувальних предметів та підсилювачів на результат
Лікувальні предмети використовуються для відновлення здоров’я після отримання поранень, що покращує стан здоров’я і дозволяє гравцю довше залишатися у грі.

Підсилювальні предмети використовуються для збільшення швидкості та точності, що дає змогу гравцю зробити більше вбивств або швидше втекти з бою.



In [ ]:
# Summary statistics for the number of healing items used
print('The average person uses {:.2f} healing items'.format(df['heals'].mean()))
print('50% of people used {:.2f} healing items'.format(df['heals'].quantile(0.50)))
print('75% of people used {:.2f} healing items or less'.format(df['heals'].quantile(0.75)))
print('99% of people used {:.2f} healing items or less'.format(df['heals'].quantile(0.99)))
print('The doctor of the data used {:.2f} healing items'.format(df['heals'].max()))

In [ ]:
# Summary statistics for the number of boosting items used
print('The average person uses {:.2f} boosting items'.format(df['boosts'].mean()))
print('50% of people used {:.2f} boosting items'.format(df['boosts'].quantile(0.50)))
print('75% of people used {:.2f} boosting items or less'.format(df['boosts'].quantile(0.75)))
print('99% of people used {:.2f} boosting items or less'.format(df['boosts'].quantile(0.99)))
print('The addict of the data used {:.2f} boosting items'.format(df['boosts'].max()))

In [ ]:
data = df.copy()
data = data[data['heals'] < data['heals'].quantile(0.99)]
data = data[data['boosts'] < data['boosts'].quantile(0.99)]

f,ax1 = plt.subplots(figsize =(20,10))
sns.pointplot(x='heals',y='winPlacePerc',data=data,color='red',alpha=1.0)
sns.pointplot(x='boosts',y='winPlacePerc',data=data,color='blue',alpha=0.8)
plt.text(4,0.6,'Heals',color='red',fontsize = 17,style = 'italic')
plt.text(4,0.55,'Boosts',color='blue',fontsize = 17,style = 'italic')
plt.xlabel('Number of heal/boost items',fontsize = 15,color='blue')
plt.ylabel('Win Percentage',fontsize = 15,color='blue')
plt.title('Heals vs Boosts',fontsize = 20,color='blue')
plt.grid()

##### Спостереження:
Тут ми можемо побачити, як використовуються лікувальні предмети (heal items) та підсилювальні предмети (boost items) у порівнянні один з одним.

Схоже, що використання невеликої кількості лікувальних предметів підвищує шанси на перемогу, але для реального впливу на результат потрібно використовувати більше підсилювачів.

### Багатовимірний аналіз


#### Кореляція Пірсона між усіма ознаками


In [ ]:
f,ax = plt.subplots(figsize=(15, 15))
sns.heatmap(df.corr(), annot=True, linewidths=.5, fmt= '.2f',ax=ax)

##### Спостереження:
Відповідно до кольорової шкали (colorbar), ми можемо визначити кореляцію між різними ознаками.

Якщо кореляція позитивна, одна змінна збільшується разом з іншою.

Якщо кореляція негативна, при збільшенні однієї змінної інша зменшується.

Якщо кореляція дорівнює 1, це означає, що змінні однакові або майже однакові.



Дякую за перегляд цього блокноту!

Будь ласка, поставте вподобайку, якщо це було корисно.